<a href="https://colab.research.google.com/github/mlaricobar/solution-datathon-interbank-2020/blob/desarrollo/%5B04%5D%20Model%20Building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CONSTRUCCIÓN DE LOS MODELOS


### Instalación de Librerías necesarias

In [1]:
!pip install catboost

     |████████████████████████████████| 66.3MB 49kB/s 



### Lectura de Datos

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
%cd /gdrive/'My Drive'/Competencias/'[02] Interbank-2020'

/gdrive/My Drive/Competencias/[02] Interbank-2020


In [4]:
!ls -lh '[01] Intermediate'

total 1.5G
-rw------- 1 root root 521M Dec 20 17:16 ds_rcc.csv
-rw------- 1 root root 185M Dec 18 16:51 ds_rcc_instit.csv
-rw------- 1 root root 204M Dec 18 16:51 ds_rcc_instit_test.csv
-rw------- 1 root root 576M Dec 20 17:18 ds_rcc_test.csv
-rw------- 1 root root  22M Dec 20 13:20 ds_se.csv
-rw------- 1 root root  24M Dec 20 13:20 ds_se_test.csv


In [5]:
import numpy as np
import pandas as pd

In [6]:
def optimizar_df(df):
  
  for col_ in df.select_dtypes(include=[int]).columns:
    df[col_] = pd.to_numeric(df[col_], downcast="unsigned")

  for col_ in df.select_dtypes(include=[float]).columns:
    df[col_] = pd.to_numeric(df[col_], downcast="float")
  
  return df

In [7]:
df_se = optimizar_df(pd.read_csv('[01] Intermediate/ds_se.csv'))
df_rcc = optimizar_df(pd.read_csv("[01] Intermediate/ds_rcc.csv"))
#df_rcc_inst = optimizar_df(pd.read_csv("[01] Intermediate/ds_rcc_instit.csv"))

df_se_test = optimizar_df(pd.read_csv('[01] Intermediate/ds_se_test.csv'))
df_rcc_test = optimizar_df(pd.read_csv("[01] Intermediate/ds_rcc_test.csv"))
#df_rcc_inst_test = optimizar_df(pd.read_csv("[01] Intermediate/ds_rcc_instit_test.csv"))

#### Consolidar Dataset

In [8]:
df_train = (df_se.merge(df_rcc, how="left", on=["key_value"])
                .reset_index(drop=True)
           )#.merge(df_rcc_inst, how="left", on=["key_value"])

df_test = (df_se_test.merge(df_rcc_test, how="left", on=["key_value"])
                     .reset_index(drop=True)
           )#.merge(df_rcc_inst_test, how="left", on=["key_value"])

In [9]:
df_train.head()

,key_value,edad,sexo,est_cvl,sit_lab,cod_ocu,ctd_hijos,flg_sin_email,ctd_veh,cod_ubi,lgr_vot,prv,dto,rgn,tip_lvledu,target,U03M_SALDO_MEAN_FOR_PRODUCTO_0_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_0_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_0_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_1_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_1_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_1_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_2_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_2_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_2_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_3_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_3_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_3_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_4_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_4_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_4_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_6_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_6_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_6_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_8_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_8_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_8_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_255_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_255_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_255_COUNT,...,U06M_MEAN_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_COUNT,U06M_SUM_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_MEAN,U06M_SUM_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_SUM,U06M_SUM_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_COUNT,U06M_COUNT_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_MEAN,U06M_COUNT_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_SUM,U06M_COUNT_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_COUNT,U06M_RECENCY_FOR_condicion_cat,U06M_LONG_RECENCY_FOR_condicion_cat,U06M_DIFF_MONTHS_FIRST_LAST_FOR_condicion_cat,U12M_SALDO_MEAN_FOR_CONDICION_CAT_0_MEAN,U12M_SALDO_MEAN_FOR_CONDICION_CAT_0_SUM,U12M_SALDO_MEAN_FOR_CONDICION_CAT_0_COUNT,U12M_SALDO_MEAN_FOR_CONDICION_CAT_00_03_MEAN,U12M_SALDO_MEAN_FOR_CONDICION_CAT_00_03_SUM,U12M_SALDO_MEAN_FOR_CONDICION_CAT_00_03_COUNT,U12M_SALDO_MEAN_FOR_CONDICION_CAT_03_06_MEAN,U12M_SALDO_MEAN_FOR_CONDICION_CAT_03_06_SUM,U12M_SALDO_MEAN_FOR_CONDICION_CAT_03_06_COUNT,U12M_SALDO_MEAN_FOR_CONDICION_CAT_06_12_MEAN,U12M_SALDO_MEAN_FOR_CONDICION_CAT_06_12_SUM,U12M_SALDO_MEAN_FOR_CONDICION_CAT_06_12_COUNT,U12M_SALDO_MEAN_FOR_CONDICION_CAT_12-32_MEAN,U12M_SALDO_MEAN_FOR_CONDICION_CAT_12-32_SUM,U12M_SALDO_MEAN_FOR_CONDICION_CAT_12-32_COUNT,U12M_SALDO_MEAN_FOR_CONDICION_CAT_>32_MEAN,U12M_SALDO_MEAN_FOR_CONDICION_CAT_>32_SUM,U12M_SALDO_MEAN_FOR_CONDICION_CAT_>32_COUNT,U12M_MEAN_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_MEAN,U12M_MEAN_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_SUM,U12M_MEAN_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_COUNT,U12M_SUM_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_MEAN,U12M_SUM_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_SUM,U12M_SUM_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_COUNT,U12M_COUNT_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_MEAN,U12M_COUNT_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_SUM,U12M_COUNT_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_COUNT,U12M_RECENCY_FOR_condicion_cat,U12M_LONG_RECENCY_FOR_condicion_cat,U12M_DIFF_MONTHS_FIRST_LAST_FOR_condicion_cat
0,0,2.016349,1,4,2,22,0.0,1,0.0,-0.070167,17,47,687,4,0,0,1.062248,3.186743,3,0.927645,0.927645,1,0.961819,2.885458,3,0.961558,2.884674,3,NaN,0.000000,0,NaN,0.000000,0,0.927880,0.927880,1,NaN,0.0,0,...,6,0.987723,5.926337,6,1.0,6,6,1,6,5,0.964685,11.576217,12,NaN,0.000000,0,NaN,0.0,0,NaN,0.0,0,NaN,0.0,0,NaN,0.0,0,0.964685,11.576217,12,0.964685,11.576217,12,1.000,12,12,1,12,11
1,1,3.000953,1,4,1,22,0.0,1,1.0,-0.738566,10,186,849,6,0,0,NaN,0.000000,0,0.927283,2.781850,3,NaN,0.000000,0,NaN,0.000000,0,0.937762,2.813286,3,NaN,0.000000,0,NaN,0.000000,0,NaN,0.0,0,...,3,0.932511,2.797534,3,1.0,3,3,1,3,2,0.932511,2.797534,3,NaN,0.000000,0,NaN,0.0,0,NaN,0.0,0,NaN,0.0,0,NaN,0.0,0,0.932511,2.797534,3,0.932511,2.797534,3,1.000,3,3,1,3,2
2,2,3.531124,0,4,1,22,0.0,1,0.0,-0.297248,15,84,1405,1,7,0,NaN,0.000000,0,0.927242,0.927242,1,NaN,0.000000,0,NaN,0.000000,0,0.995254,0.995254,1,NaN,0.000000,0,NaN,0.000000,0,NaN,0.0,0,...,1,0.949872,0.949872,1,1.0,1,1,1,1,0,0.939484,1.878969,2,NaN,0.000000,0,NaN,0.0,0,NaN,0.0,0,NaN,0.0,0,NaN,0.0,0,0.939484,1.878969,2,0.939484,1.878969,2,1.000,2,2,1,12,11
3,3,1.864871,0,4,1,22,0.0,1,0.0,-1.203435,4,9,977,6,7,0,NaN,0.000000,0,0.9272

In [10]:
df_train.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358428 entries, 0 to 358427
Columns: 358 entries, key_value to U12M_DIFF_MONTHS_FIRST_LAST_FOR_condicion_cat
dtypes: float32(205), uint16(1), uint32(1), uint8(151)
memory usage: 334.0 MB


In [11]:
df_test.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396609 entries, 0 to 396608
Columns: 357 entries, key_value to U12M_DIFF_MONTHS_FIRST_LAST_FOR_condicion_cat
dtypes: float32(205), uint16(1), uint32(1), uint8(150)
memory usage: 369.2 MB


In [12]:
for col_ in df_train.columns:
  if df_train.loc[df_train[col_] == np.inf, :].shape[0] > 0:
    print(col_)

for col_ in df_test.columns:
  if df_test.loc[df_test[col_] == np.inf, :].shape[0] > 0:
    print(col_)

In [13]:
avoid_columns = []
main_columns = ["key_value", "target"]
predictores = [c for c in df_train.columns if c not in (main_columns + avoid_columns)]

In [14]:
print(predictores)

['edad', 'sexo', 'est_cvl', 'sit_lab', 'cod_ocu', 'ctd_hijos', 'flg_sin_email', 'ctd_veh', 'cod_ubi', 'lgr_vot', 'prv', 'dto', 'rgn', 'tip_lvledu', 'U03M_SALDO_MEAN_FOR_PRODUCTO_0_MEAN', 'U03M_SALDO_MEAN_FOR_PRODUCTO_0_SUM', 'U03M_SALDO_MEAN_FOR_PRODUCTO_0_COUNT', 'U03M_SALDO_MEAN_FOR_PRODUCTO_1_MEAN', 'U03M_SALDO_MEAN_FOR_PRODUCTO_1_SUM', 'U03M_SALDO_MEAN_FOR_PRODUCTO_1_COUNT', 'U03M_SALDO_MEAN_FOR_PRODUCTO_2_MEAN', 'U03M_SALDO_MEAN_FOR_PRODUCTO_2_SUM', 'U03M_SALDO_MEAN_FOR_PRODUCTO_2_COUNT', 'U03M_SALDO_MEAN_FOR_PRODUCTO_3_MEAN', 'U03M_SALDO_MEAN_FOR_PRODUCTO_3_SUM', 'U03M_SALDO_MEAN_FOR_PRODUCTO_3_COUNT', 'U03M_SALDO_MEAN_FOR_PRODUCTO_4_MEAN', 'U03M_SALDO_MEAN_FOR_PRODUCTO_4_SUM', 'U03M_SALDO_MEAN_FOR_PRODUCTO_4_COUNT', 'U03M_SALDO_MEAN_FOR_PRODUCTO_6_MEAN', 'U03M_SALDO_MEAN_FOR_PRODUCTO_6_SUM', 'U03M_SALDO_MEAN_FOR_PRODUCTO_6_COUNT', 'U03M_SALDO_MEAN_FOR_PRODUCTO_8_MEAN', 'U03M_SALDO_MEAN_FOR_PRODUCTO_8_SUM', 'U03M_SALDO_MEAN_FOR_PRODUCTO_8_COUNT', 'U03M_SALDO_MEAN_FOR_PRODUCTO_255

### Preparar Marco de Evaluación

In [15]:
nrows, nrows_train, nrows_test = (df_train.shape[0] + df_test.shape[0]), df_train.shape[0], df_test.shape[0]

print("Cantidad de filas: {0:,}\n\t- Train:\t{1:,} filas\t({2:}%)\n\t- Test:\t\t{3:,} filas\t({4:}%)".format(nrows,
                                                                                                            nrows_train,
                                                                                                            round((100 * (nrows_train/nrows)), 2),
                                                                                                            nrows_test,
                                                                                                            round((100 * (nrows_test/nrows)), 2)
                                                                                                            ))

Cantidad de filas: 755,037
	- Train:	358,428 filas	(47.47%)
	- Test:		396,609 filas	(52.53%)


In [16]:
df_train.head()

,key_value,edad,sexo,est_cvl,sit_lab,cod_ocu,ctd_hijos,flg_sin_email,ctd_veh,cod_ubi,lgr_vot,prv,dto,rgn,tip_lvledu,target,U03M_SALDO_MEAN_FOR_PRODUCTO_0_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_0_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_0_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_1_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_1_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_1_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_2_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_2_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_2_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_3_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_3_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_3_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_4_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_4_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_4_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_6_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_6_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_6_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_8_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_8_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_8_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_255_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_255_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_255_COUNT,...,U06M_MEAN_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_COUNT,U06M_SUM_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_MEAN,U06M_SUM_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_SUM,U06M_SUM_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_COUNT,U06M_COUNT_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_MEAN,U06M_COUNT_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_SUM,U06M_COUNT_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_COUNT,U06M_RECENCY_FOR_condicion_cat,U06M_LONG_RECENCY_FOR_condicion_cat,U06M_DIFF_MONTHS_FIRST_LAST_FOR_condicion_cat,U12M_SALDO_MEAN_FOR_CONDICION_CAT_0_MEAN,U12M_SALDO_MEAN_FOR_CONDICION_CAT_0_SUM,U12M_SALDO_MEAN_FOR_CONDICION_CAT_0_COUNT,U12M_SALDO_MEAN_FOR_CONDICION_CAT_00_03_MEAN,U12M_SALDO_MEAN_FOR_CONDICION_CAT_00_03_SUM,U12M_SALDO_MEAN_FOR_CONDICION_CAT_00_03_COUNT,U12M_SALDO_MEAN_FOR_CONDICION_CAT_03_06_MEAN,U12M_SALDO_MEAN_FOR_CONDICION_CAT_03_06_SUM,U12M_SALDO_MEAN_FOR_CONDICION_CAT_03_06_COUNT,U12M_SALDO_MEAN_FOR_CONDICION_CAT_06_12_MEAN,U12M_SALDO_MEAN_FOR_CONDICION_CAT_06_12_SUM,U12M_SALDO_MEAN_FOR_CONDICION_CAT_06_12_COUNT,U12M_SALDO_MEAN_FOR_CONDICION_CAT_12-32_MEAN,U12M_SALDO_MEAN_FOR_CONDICION_CAT_12-32_SUM,U12M_SALDO_MEAN_FOR_CONDICION_CAT_12-32_COUNT,U12M_SALDO_MEAN_FOR_CONDICION_CAT_>32_MEAN,U12M_SALDO_MEAN_FOR_CONDICION_CAT_>32_SUM,U12M_SALDO_MEAN_FOR_CONDICION_CAT_>32_COUNT,U12M_MEAN_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_MEAN,U12M_MEAN_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_SUM,U12M_MEAN_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_COUNT,U12M_SUM_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_MEAN,U12M_SUM_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_SUM,U12M_SUM_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_COUNT,U12M_COUNT_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_MEAN,U12M_COUNT_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_SUM,U12M_COUNT_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT_COUNT,U12M_RECENCY_FOR_condicion_cat,U12M_LONG_RECENCY_FOR_condicion_cat,U12M_DIFF_MONTHS_FIRST_LAST_FOR_condicion_cat
0,0,2.016349,1,4,2,22,0.0,1,0.0,-0.070167,17,47,687,4,0,0,1.062248,3.186743,3,0.927645,0.927645,1,0.961819,2.885458,3,0.961558,2.884674,3,NaN,0.000000,0,NaN,0.000000,0,0.927880,0.927880,1,NaN,0.0,0,...,6,0.987723,5.926337,6,1.0,6,6,1,6,5,0.964685,11.576217,12,NaN,0.000000,0,NaN,0.0,0,NaN,0.0,0,NaN,0.0,0,NaN,0.0,0,0.964685,11.576217,12,0.964685,11.576217,12,1.000,12,12,1,12,11
1,1,3.000953,1,4,1,22,0.0,1,1.0,-0.738566,10,186,849,6,0,0,NaN,0.000000,0,0.927283,2.781850,3,NaN,0.000000,0,NaN,0.000000,0,0.937762,2.813286,3,NaN,0.000000,0,NaN,0.000000,0,NaN,0.0,0,...,3,0.932511,2.797534,3,1.0,3,3,1,3,2,0.932511,2.797534,3,NaN,0.000000,0,NaN,0.0,0,NaN,0.0,0,NaN,0.0,0,NaN,0.0,0,0.932511,2.797534,3,0.932511,2.797534,3,1.000,3,3,1,3,2
2,2,3.531124,0,4,1,22,0.0,1,0.0,-0.297248,15,84,1405,1,7,0,NaN,0.000000,0,0.927242,0.927242,1,NaN,0.000000,0,NaN,0.000000,0,0.995254,0.995254,1,NaN,0.000000,0,NaN,0.000000,0,NaN,0.0,0,...,1,0.949872,0.949872,1,1.0,1,1,1,1,0,0.939484,1.878969,2,NaN,0.000000,0,NaN,0.0,0,NaN,0.0,0,NaN,0.0,0,NaN,0.0,0,0.939484,1.878969,2,0.939484,1.878969,2,1.000,2,2,1,12,11
3,3,1.864871,0,4,1,22,0.0,1,0.0,-1.203435,4,9,977,6,7,0,NaN,0.000000,0,0.9272

### Construcción de Modelos

In [17]:
import lightgbm as lgb
from catboost import CatBoostClassifier, CatBoostRegressor

from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold

from scipy.optimize import differential_evolution

In [18]:
seed_val = 41

ctb_params = {'loss_function':'Logloss',
              'eval_metric':'AUC',
              'task_type': 'GPU',
              'verbose': 200,
              'random_seed': 1
             }

lgb_params = {
                "objective": "binary",
                "metric": "auc",
                "max_depth": 8,
                "learning_rate": 0.05,
                "bagging_seed": 41,
                "min_data_in_leaf": 20,
                "bagging_fraction": 0.7,
                "feature_fraction": 0.7,
                "bagging_freq": 5,
                "verbosity": -1
              }

In [19]:
k = 5 # n folds

def model_with_cv(df_train, df_test, target_class, predictores, df_val=None):

  ##################################################################
  ############# ETAPA DE PREDICCIÓN
  ##################################################################

  # Se instancian las variables que se obtendrán como resultado.
  results_models = []
  list_models_cv = []

  df_probs_train = df_train[["key_value", target_class]].copy().reset_index(drop=True)
  df_probs_test = df_test[["key_value"]].copy().reset_index(drop=True)

  if df_val is not None:
    df_probs_val = df_val[["key_value", target_class]].copy().reset_index(drop=True)
  else:
    df_probs_val = None
  # Generar los Folds
  skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=0)

  for i, (traincv, valcv) in enumerate(skf.split(df_train[predictores], df_train[target_class])):

      print("Iteracion {}".format(i))
      
      df_train_cv = df_train.loc[traincv, :].copy().reset_index(drop=True)
      df_val_cv = df_train.loc[valcv, :].copy().reset_index(drop=True)

      # Fase de Entrenamiento
      print("\t CLASIFICACIÓN")
      print("\t\t 1. CATBOOST")
      ## CatBoost
      cbt_model = CatBoostClassifier(**ctb_params)
      cbt_model.fit(df_train_cv[predictores], df_train_cv[target_class], eval_set=(df_val_cv[predictores], df_val_cv[target_class]), use_best_model=True)

      print("\t\t 2. LIGHTGBM")
      ## LightGBM
      lgtrain = lgb.Dataset(df_train_cv[predictores], label=df_train_cv[target_class])
      lgval = lgb.Dataset(df_val_cv[predictores], label=df_val_cv[target_class])
      lgb_model = lgb.train(lgb_params, lgtrain, 900, valid_sets=[lgtrain, lgval], early_stopping_rounds=100, verbose_eval=20, evals_result={})

      # Fase de Scoring
      ## Clasificación
      probas_train_cbtm = cbt_model.predict_proba(df_train_cv[predictores])[:, 1]
      probas_val_cbtm = cbt_model.predict_proba(df_val_cv[predictores])[:, 1]

      probas_train_lgbm = lgb_model.predict(df_train_cv[predictores])
      probas_val_lgbm = lgb_model.predict(df_val_cv[predictores])

      # Cálculo de Métricas
      ## Clasificación
      metric_train_cbtm = roc_auc_score(df_train_cv[target_class], probas_train_cbtm)
      metric_val_cbtm = roc_auc_score(df_val_cv[target_class], probas_val_cbtm)

      metric_train_lgbm = roc_auc_score(df_train_cv[target_class], probas_train_lgbm)
      metric_val_lgbm = roc_auc_score(df_val_cv[target_class], probas_val_lgbm)

      if df_val is not None:
        probas_val_out_cbtm = cbt_model.predict_proba(df_val[predictores])[:, 1]
        metric_val_out_cbtm = roc_auc_score(df_val[target_class], probas_val_out_cbtm)

        probas_val_out_lgbm = lgb_model.predict(df_val[predictores])
        metric_val_out_lgbm = roc_auc_score(df_val[target_class], probas_val_out_lgbm)

      else:
        probas_val_out_cbtm = np.zeros((1,1))
        metric_val_out_cbtm = 0

        probas_val_out_lgbm = np.zeros((1,1))
        metric_val_out_lgbm = 0

      # Cálculo de probabilidades en la base de Train y Validacion
      df_probs_train.loc[valcv, "prob_cbtm"] = probas_val_cbtm
      df_probs_train.loc[valcv, "prob_lgbm"] = probas_val_lgbm
      df_probs_train.loc[valcv, "FOLD"] = "FOLD_{0}".format(i+1)

      
      df_probs_test["prob_cbtm_{}".format(i)] = cbt_model.predict_proba(df_test[predictores])[:, 1]
      df_probs_test["prob_lgbm_{}".format(i)] = lgb_model.predict(df_test[predictores])

      if df_val is not None:
        df_probs_val["prob_cbtm_{}".format(i)] = cbt_model.predict_proba(df_val[predictores])[:, 1]
        df_probs_val["prob_lgbm_{}".format(i)] = lgb_model.predict(df_val[predictores])

        results_models.append({"auc_train_cbtm": metric_train_cbtm, "auc_val_cbtm": metric_val_cbtm, "auc_val_out_cbtm": metric_val_out_cbtm, ## CATBOOST
                          "auc_train_lgbm": metric_train_lgbm, "auc_val_lgbm": metric_val_lgbm, "auc_val_out_lgbm": metric_val_out_lgbm, ## LIGHTGBM
                          "seed_val": seed_val})
        
        print("AVG METRICS CLASSIFICATION")
        print("Mean Target:\tTrain: {0},\tVal In: {1},\tVal Out: {2}".format(round(df_train_cv[target_class].mean(),4), round(df_val_cv[target_class].mean(),4), round(df_val[target_class].mean(),4)))
        print("Mean Probs:\tTrain: {0},\tVal In: {1},\tVal Out: {2}".format(round(np.mean(probas_train_cbtm),4), round(np.mean(probas_val_cbtm),4), round(np.mean(probas_val_out_cbtm),4)))
        print("Std Probs:\tTrain: {0},\tVal In: {1},\tVal Out: {2}".format(round(np.std(probas_train_cbtm),4), round(np.std(probas_val_cbtm),4), round(np.std(probas_val_out_cbtm),4)))

      else:

        # Almacenar los resultados
        results_models.append({"auc_train_cbtm": metric_train_cbtm, "auc_val_cbtm": metric_val_cbtm, ## CATBOOST
                          "auc_train_lgbm": metric_train_lgbm, "auc_val_lgbm": metric_val_lgbm, ## LIGHTGBM
                          "seed_val": seed_val})
        
        print("AVG METRICS CLASSIFICATION")
        print("Mean Target:\tTrain: {0},\tVal In: {1}".format(round(df_train_cv[target_class].mean(),4), round(df_val_cv[target_class].mean(),4)))
        print("Mean Probs:\tTrain: {0},\tVal In: {1}".format(round(np.mean(probas_train_cbtm),4), round(np.mean(probas_val_cbtm),4)))
        print("Std Probs:\tTrain: {0},\tVal In: {1}".format(round(np.std(probas_train_cbtm),4), round(np.std(probas_val_cbtm),4)))
        
        
      list_models_cv.append({"cbtm_model": cbt_model, "lgbm_model": lgb_model})
      
      print("CATBOOST REPORT")
      if df_val is not None:
        df_prob_target_tmp = pd.DataFrame({"prob": probas_val_out_cbtm, "target": df_val[target_class]})
      else:
        df_prob_target_tmp = pd.DataFrame({"prob": probas_val_cbtm, "target": df_val_cv[target_class]})

      df_prob_target_tmp = df_prob_target_tmp.groupby(pd.qcut(df_prob_target_tmp["prob"], q=10)).agg({"target": ["sum", "count", "mean"]}).reset_index(drop=False)
      df_prob_target_tmp.columns = ["_".join(c) if c[-1] != "" else c[0] for c in df_prob_target_tmp.columns]
      df_prob_target_tmp["target_%"] = round(df_prob_target_tmp["target_sum"] * 100.0 / df_prob_target_tmp["target_sum"].sum(), 2)
      print(df_prob_target_tmp)
      print("#" * 500)

      print("LIGHTGBM REPORT")
      if df_val is not None:
        df_prob_target_tmp = pd.DataFrame({"prob": probas_val_out_lgbm, "target": df_val[target_class]})
      else:
        df_prob_target_tmp = pd.DataFrame({"prob": probas_val_lgbm, "target": df_val_cv[target_class]})

      df_prob_target_tmp = df_prob_target_tmp.groupby(pd.qcut(df_prob_target_tmp["prob"], q=10)).agg({"target": ["sum", "count", "mean"]}).reset_index(drop=False)
      df_prob_target_tmp.columns = ["_".join(c) if c[-1] != "" else c[0] for c in df_prob_target_tmp.columns]
      df_prob_target_tmp["target_%"] = round(df_prob_target_tmp["target_sum"] * 100.0 / df_prob_target_tmp["target_sum"].sum(), 2)
      print(df_prob_target_tmp)
      print("#" * 500)
  
  ##################################################################
  ############# PARA ETAPA DE OPTIMIZACIÓN
  ##################################################################

  if df_val is not None:
    df_probs_val["prob_avg_cbtm"] = df_probs_val[[c for c in df_probs_val.columns if "prob_cbtm" in c]].mean(axis=1)
    df_probs_val["prob_avg_lgbm"] = df_probs_val[[c for c in df_probs_val.columns if "prob_lgbm" in c]].mean(axis=1)

  df_probs_test["prob_avg_cbtm"] = df_probs_test[[c for c in df_probs_test.columns if "prob_cbtm" in c]].mean(axis=1)
  df_probs_test["prob_avg_lgbm"] = df_probs_test[[c for c in df_probs_test.columns if "prob_lgbm" in c]].mean(axis=1)

  
  return results_models, list_models_cv, df_probs_train, df_probs_val, df_probs_test

In [20]:
results, list_models, df_probs_train, df_probs_val, df_probs_test = model_with_cv(df_train, df_test, "target", predictores, None)

Iteracion 0
	 CLASIFICACIÓN
		 1. CATBOOST
Learning rate set to 0.044019
0:	learn: 0.7637732	test: 0.7660027	best: 0.7660027 (0)	total: 18.2ms	remaining: 18.1s
200:	learn: 0.8313764	test: 0.8310706	best: 0.8310706 (200)	total: 3.19s	remaining: 12.7s
400:	learn: 0.8407434	test: 0.8374161	best: 0.8374161 (400)	total: 6.24s	remaining: 9.32s
600:	learn: 0.8464025	test: 0.8402108	best: 0.8402108 (600)	total: 9.34s	remaining: 6.2s
800:	learn: 0.8505939	test: 0.8416989	best: 0.8416989 (800)	total: 12.4s	remaining: 3.08s
999:	learn: 0.8543731	test: 0.8427482	best: 0.8427482 (999)	total: 15.4s	remaining: 0us
bestTest = 0.8427482247
bestIteration = 999
		 2. LIGHTGBM
Training until validation scores don't improve for 100 rounds.
[20]	training's auc: 0.807554	valid_1's auc: 0.807744
[40]	training's auc: 0.821272	valid_1's auc: 0.820575
[60]	training's auc: 0.829305	valid_1's auc: 0.827372
[80]	training's auc: 0.835923	valid_1's auc: 0.832452
[100]	training's auc: 0.840076	valid_1's auc: 0.835307


In [ ]:
# First Version
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.834539,0.822712,0.859006,0.823846,41
1,0.835161,0.820945,0.855816,0.821616,41
2,0.835441,0.817861,0.857260,0.819050,41
3,0.835025,0.821256,0.851961,0.822491,41
4,0.835012,0.819481,0.872939,0.821429,41


In [ ]:
# Second Version
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.843651,0.829637,0.890753,0.831454,41
1,0.844369,0.827960,0.869049,0.829013,41
2,0.844213,0.825024,0.880458,0.827586,41
3,0.843544,0.828033,0.883655,0.830802,41
4,0.843915,0.827735,0.890578,0.830337,41


In [ ]:
# Third Version
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.837454,0.822779,0.875155,0.824367,41
1,0.837709,0.822735,0.871554,0.823324,41
2,0.838256,0.819201,0.881905,0.820629,41
3,0.837436,0.822897,0.872295,0.824793,41
4,0.837011,0.822719,0.881190,0.824380,41


In [ ]:
# Fourth Version
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.848285,0.833916,0.898653,0.836538,41
1,0.848607,0.832712,0.890521,0.834790,41
2,0.848699,0.830145,0.891555,0.832881,41
3,0.848064,0.831962,0.897827,0.835895,41
4,0.848851,0.831197,0.898466,0.833766,41


In [22]:
# Fourth Version
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.854373,0.842748,0.890797,0.844431,41
1,0.855238,0.840793,0.892966,0.842644,41
2,0.855101,0.836793,0.903635,0.840386,41
3,0.854775,0.839483,0.902767,0.842496,41
4,0.855069,0.839889,0.903772,0.841770,41


### Importancia de Variables

In [ ]:
df_predictores_ = pd.DataFrame({"name": predictores, "importance_fold_0": list_models[0]["cbtm_model"].get_feature_importance()})
for i in range(1, 5):
    df_predictores_ = df_predictores_.merge(pd.DataFrame({"name": predictores, "importance_fold_{0}".format(i): list_models[i]["cbtm_model"].get_feature_importance()}), how="left", on="name")

df_predictores_["importance_folds_mean"] = df_predictores_[[c for c in df_predictores_.columns if c not in ["name"]]].mean(axis=1)
df_predictores_ = df_predictores_.sort_values(by="importance_folds_mean", ascending=False).reset_index(drop=True)
df_predictores_.head(30)

,name,importance_fold_0,importance_fold_1,importance_fold_2,importance_fold_3,importance_fold_4,importance_folds_mean
0,U03M_SALDO_MEAN_FOR_PRODUCTO_TOP_PROPENSITY_COUNT,10.926812,11.683049,12.745838,12.187827,12.527125,12.014130
1,U03M_SALDO_MEAN_FOR_PRODUCTO_6_MEAN,6.755853,6.895747,7.177519,7.253558,7.197517,7.056039
2,U03M_SALDO_MEAN_FOR_PRODUCTO_TOP_PROPENSITY_MEAN,4.946537,4.446514,3.426246,5.072011,3.228418,4.223945
3,U03M_SALDO_MEAN_FOR_PRODUCTO_1_MEAN,3.899354,3.841197,3.815469,3.676805,3.756662,3.797897
4,U03M_SALDO_MEAN_FOR_PRODUCTO_3_MEAN,3.010013,2.565782,2.468228,2.835244,2.941321,2.764118
5,U03M_SALDO_MEAN_FOR_PRODUCTO_0_SUM,2.687775,2.625506,2.284568,2.388692,2.261344,2.449577
6,U03M_SUM_OF_SALDO_MEAN_FOR_ALL_PRODUCTO_MEAN,2.182451,2.152685,1.806553,2.348395,2.836498,2.265316
7,tip_lvledu,1.997374,2.015725,1.994719,2.233607,2.296822,2.107649
8,U06M_SALDO_MEAN_FOR_PRODUCTO_6_MEAN,2.150350,2.126063,2.096997,1.909200,2.005934,2.057709
9,U03M_MEAN_OF_SALDO_MEAN_FOR_ALL_COD_INSTIT_FIN...,2.055891,2.032803,1.640202,1.825735,2.370639,1.985054


### Estabilidad de las Variables

In [ ]:
def func_var_porc(n, d):
  try:
    return round(100.0 * (n/d - 1), 2)
  except:
    return None

In [ ]:
df_predictores_["train_mean"] = df_predictores_["name"].apply(lambda c: df_train[c].mean())
df_predictores_["train_std"] = df_predictores_["name"].apply(lambda c: df_train[c].std())
df_predictores_["test_mean"] = df_predictores_["name"].apply(lambda c: df_test[c].mean())
df_predictores_["test_std"] = df_predictores_["name"].apply(lambda c: df_test[c].std())

df_predictores_["var_porc_mean_test_train"] = df_predictores_.apply(lambda row: func_var_porc(row["test_mean"], row["train_mean"]), axis=1)
df_predictores_["var_porc_std_test_train"] = df_predictores_.apply(lambda row: func_var_porc(row["test_std"], row["train_std"]), axis=1)

df_predictores_["abs_var_porc_mean_test_train"] = abs(df_predictores_["var_porc_mean_test_train"])

In [ ]:
df_predictores_.shape, df_predictores_.loc[df_predictores_["abs_var_porc_mean_test_train"] < 10].shape

((149, 14), (104, 14))

In [ ]:
new_predictores = df_predictores_.loc[df_predictores_["abs_var_porc_mean_test_train"] < 10, "name"].tolist()

### Interpretación

### Generando Submit

In [ ]:
df_probs_test[["key_value", "prob_avg_cbtm"]].rename(columns={"prob_avg_cbtm": "target"}).to_csv('[02] Output/submit_01.csv', index=False)

In [ ]:
!ls -lh '[02] Output'

total 11M
-rw------- 1 root root 11M Dec 17 21:03 submit_01.csv


------